### Recogida y envío de datos del día actual de Consumo y Generación de 8 Edificios de la ULL a ThingsBoard

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time # Para trabajar con fechas y tiempo
import os # Librería que nos permite interactuar con el sistema de archivos
import glob # para buscar archivos en un directorio
import pandas as pd
import requests # Para realizar solicitud HTTP con ThingsBoard 
import json # Formato al que hay que enviar los datos a la platafor IoT ThinsBoard

def inicializar_navegador():
    options = webdriver.ChromeOptions()  # Opciones de navegación con selenium

    # Configuraciones de descarga en modo headless
    options.add_argument("--headless")  
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920x1080")

    prefs = {
        "download.default_directory": download_folder,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "safebrowsing.disable_download_protection": True,
        "profile.default_content_setting_values.automatic_downloads": 1, # Permitir múltiples descargas
    }

    options.add_experimental_option("prefs", prefs)

    # Para asegurarse de que no se bloqueen las descargas en modo headless
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    # Inicializar el driver con las opciones configuradas
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    return driver

# Función para verificar la descarga completa del archivo y guardarlo como el archivo más reciente
def esperar_descarga_completa(download_folder, extension="xls", timeout=60):
    start_time = time.time()
    while True:
        files = glob.glob(download_folder + f"/*.{extension}")
        if files:
            latest_file = max(files, key=os.path.getctime)  # Obtener el archivo más reciente
            print("Archivo descargado con éxito: " + latest_file)
            print()
            return latest_file
        elif time.time() - start_time > timeout:
            print("Tiempo de espera para la descarga excedido.")
            print()
            return None
        time.sleep(1)


# Función para enviar datos a ThingsBoard
def enviar_a_thingsboard(archivo_reciente, token):
    # URL del Sevirdor IAAS y Token del Dispositivo en ThinsBoard
    url = f"https://greenenergy.iaas.ull.es:8080/api/v1/{token}/telemetry"

    # Leer el archivo excel
    df = pd.read_excel(archivo_reciente, header=2)
    # Inicializar el contador de datos enviados exitosamente
    envios_exitosos = 0
    # Iterar sobre cada fila del DataFrame y enviar los datos a ThingsBoard
    for index, row in df.iterrows():
        # Construir el objeto de datos con las filas del DataFrame y el tiempo
        data = {
            "ts": pd.to_datetime(row['Time']).timestamp() * 1000, # Formato UNIX
            "values": {
                "PV(W)": row['PV(W)'],
                "Meter(W)": row['Meter(W)'],
                "Load(W)": row['Load(W)']
            }
        }
        headers = {'Content-Type': 'application/json', 'X-Authorization': f"Bearer {token}"}
        response = requests.post(url, data=json.dumps(data), headers=headers)
        # Verificación de datos enviados correctamente
        if response.status_code == 200:
            print(f"Dato enviado correctamente: {data}")
            envios_exitosos += 1
        else:
            print(f"Error al enviar dato: {response.text}")
    
    # Si los datos fueron enviados correctamente, borramos archivo de carpeta de origen
    if envios_exitosos == len(df):
        print()
        print("Todos los datos han sido enviados a ThingsBoard exitosamente. Eliminando archivo...")
        os.remove(archivo_reciente)
        print("Archivo eliminado con éxito.")
    else:
        print("Algunos datos no se pudieron enviar correctamente. El archivo se eliminará igualmente.")
        os.remove(archivo_reciente)
        print()

def iniciar_sesion_y_descargar(driver, download_folder, edificio):
    # Inicio de sesión y descarga de datos de Sems Portal
    login_url = "https://www.semsportal.com/home/login"
    driver.get(login_url)

    # Completar el formulario de inicio de sesión
    time.sleep(2)  # Esperar a que la página cargue

    # Esperar a que la página cargue y localizar la casilla de condiciones de uso y política de privacidad
    checkbox = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "readStatement"))
    )

    # Hacer clic en la casilla de verificación si no está ya seleccionada
    if not checkbox.is_selected():
        checkbox.click()

    email_input = driver.find_element(By.ID, "username")
    password_input = driver.find_element(By.ID, "password")
    login_button = driver.find_element(By.ID, "btnLogin")

    # Credenciales
    your_email = "******************"
    your_password = "********"

    email_input.send_keys(your_email)
    password_input.send_keys(your_password)
    login_button.click()

    time.sleep(10)  # Esperar a que la página cargue

    # Navegar al enlace específico del edificio
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//span[@title='{edificio['nombre']}']"))
    ).click()

    time.sleep(5)  # Esperar a que todos los elementos se carguen completamente


    # Navegar a la URL específica del Edificio
    driver.get(edificio['url'])
    time.sleep(3)  # Espera para asegurar que la página ha cargado completamente

    max_retries = 3 # Intentos de descarga de un archivo
    archivo_descargado = False  # Indica si el archivo ha sido descargado correctamente

    # Proceso de descarga de datos
    for attempt in range(max_retries):
        try:
            WebDriverWait(driver, 20).until(EC.invisibility_of_element((By.CSS_SELECTOR, '.el-loading-mask')))
            WebDriverWait(driver, 20).until(EC.invisibility_of_element((By.CSS_SELECTOR, '.el-loading-spinner')))
            action = ActionChains(driver)
            export_icon = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".goodwe-station-charts__export.fr"))
            )
            action.move_to_element(export_icon).click().perform()

            archivo_reciente = esperar_descarga_completa(download_folder=download_folder, extension="xls")
            if archivo_reciente:
                print(f"Clic en el icono de exportación realizado con éxito y archivo descargado.")
                enviar_a_thingsboard(archivo_reciente, edificio['token'])
                archivo_descargado = True
                break  # Sale del bucle de reintentos si se descarga el archivo con éxito
            else:
                print(f"Intento {attempt + 1} de descarga fallido. Reintentando...")
        except Exception as e:
            print(f"Ocurrió un error durante el intento {attempt + 1}: {e}")
        time.sleep(5)  # Espera antes de reintentar

    if not archivo_descargado:
        print("La descarga del archivo falló después de varios intentos.")

# Carpeta de donde se guardará el archivo con los datos
download_folder = "C:\\Users\\angel\\Desktop\\MASTER\\TFM"

# Lista de edificios, sus tokens y URL
edificios_tokens = [
    {"nombre": "Aparejadores", "token": "6q9h9xsn20pmr2ipv7v8", "url": "https://www.semsportal.com/powerstation/PowerStatusSnMin/cf4818e8-f972-440d-919d-2eefd176fdf9"},
    {"nombre": "BIOLOGÍA - ULL", "token": "itluv87kt3aw3uakrmoy", "url": "https://www.semsportal.com/powerstation/PowerStatusSnMin/6012db63-bcc1-4c85-84d1-373c87c16838"},
    {"nombre": "Bellas Artes", "token": "favpht3sbbkn6qi7ewpj", "url": "https://www.semsportal.com/powerstation/PowerStatusSnMin/641e57d7-2762-4d6a-9f02-4fa9049539f3"},
    {"nombre": "Derecho", "token": "2dw98a45hkcy3wp69ajf", "url": "https://www.semsportal.com/powerstation/PowerStatusSnMin/b2c6d945-2a31-4a26-861c-f0c83890d691"},
    {"nombre": "ESTABULARIO - ULL", "token": "oYy3f5Ucmn5C7yx0ehxE", "url": "https://www.semsportal.com/powerstation/PowerStatusSnMin/a55500ef-bf7e-4847-99a9-3e3f5d336e95"},
    {"nombre": "Periodismo", "token": "jfio8213xlct21c83xcd", "url": "https://www.semsportal.com/powerstation/PowerStatusSnMin/3bf6f983-4566-4018-8c5f-25aad14464eb"},
    {"nombre": "SEGAI - ULL", "token": "1s32b4n5k9bribml4yqs", "url": "https://www.semsportal.com/powerstation/PowerStatusSnMin/2c6eb65e-dca9-4c6f-810f-769064cc6ca8"}
]

for edificio in edificios_tokens:
    driver = inicializar_navegador()
    try:
        iniciar_sesion_y_descargar(driver, download_folder, edificio)
    finally:
        driver.quit()  # Cerramos el navegador 

Archivo descargado con éxito: C:\Users\angel\Desktop\MASTER\TFM\Plant Power_20240404202130.xls

Clic en el icono de exportación realizado con éxito y archivo descargado.
Dato enviado correctamente: {'ts': 1712185200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -2481, 'Load(W)': 2481}}
Dato enviado correctamente: {'ts': 1712185500000.0, 'values': {'PV(W)': 0, 'Meter(W)': -3594, 'Load(W)': 3594}}
Dato enviado correctamente: {'ts': 1712185800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -2838, 'Load(W)': 2838}}
Dato enviado correctamente: {'ts': 1712186100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -2835, 'Load(W)': 2835}}
Dato enviado correctamente: {'ts': 1712186400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -3774, 'Load(W)': 3774}}
Dato enviado correctamente: {'ts': 1712186700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -2487, 'Load(W)': 2487}}
Dato enviado correctamente: {'ts': 1712187000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -2838, 'Load(W)': 2838}}
Dato enviado correctamente: {'ts': 17121873000

Dato enviado correctamente: {'ts': 1712206800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -3120, 'Load(W)': 3120}}
Dato enviado correctamente: {'ts': 1712207100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -4203, 'Load(W)': 4203}}
Dato enviado correctamente: {'ts': 1712207400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -3249, 'Load(W)': 3249}}
Dato enviado correctamente: {'ts': 1712207700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -3282, 'Load(W)': 3282}}
Dato enviado correctamente: {'ts': 1712208000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -4674, 'Load(W)': 4674}}
Dato enviado correctamente: {'ts': 1712208300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -3594, 'Load(W)': 3594}}
Dato enviado correctamente: {'ts': 1712208600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -4392, 'Load(W)': 4392}}
Dato enviado correctamente: {'ts': 1712208900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -7221, 'Load(W)': 7221}}
Dato enviado correctamente: {'ts': 1712209200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -5865, 'Load(W)':

Dato enviado correctamente: {'ts': 1712228400000.0, 'values': {'PV(W)': 17624, 'Meter(W)': -3443, 'Load(W)': 21067}}
Dato enviado correctamente: {'ts': 1712228700000.0, 'values': {'PV(W)': 27141, 'Meter(W)': -1140, 'Load(W)': 28281}}
Dato enviado correctamente: {'ts': 1712229000000.0, 'values': {'PV(W)': 19824, 'Meter(W)': -3778, 'Load(W)': 23602}}
Dato enviado correctamente: {'ts': 1712229300000.0, 'values': {'PV(W)': 18343, 'Meter(W)': 3201, 'Load(W)': 15142}}
Dato enviado correctamente: {'ts': 1712229600000.0, 'values': {'PV(W)': 18192, 'Meter(W)': 5823, 'Load(W)': 12369}}
Dato enviado correctamente: {'ts': 1712229900000.0, 'values': {'PV(W)': 20692, 'Meter(W)': 3912, 'Load(W)': 16780}}
Dato enviado correctamente: {'ts': 1712230200000.0, 'values': {'PV(W)': 19364, 'Meter(W)': 3840, 'Load(W)': 15524}}
Dato enviado correctamente: {'ts': 1712230500000.0, 'values': {'PV(W)': 17443, 'Meter(W)': 4905, 'Load(W)': 12538}}
Dato enviado correctamente: {'ts': 1712230800000.0, 'values': {'PV(W)

Dato enviado correctamente: {'ts': 1712201100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -25191, 'Load(W)': 25191}}
Dato enviado correctamente: {'ts': 1712201400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -28494, 'Load(W)': 28494}}
Dato enviado correctamente: {'ts': 1712201700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -24850, 'Load(W)': 24850}}
Dato enviado correctamente: {'ts': 1712202000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -25080, 'Load(W)': 25080}}
Dato enviado correctamente: {'ts': 1712202300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -24550, 'Load(W)': 24550}}
Dato enviado correctamente: {'ts': 1712202600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -29326, 'Load(W)': 29326}}
Dato enviado correctamente: {'ts': 1712202900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30664, 'Load(W)': 30664}}
Dato enviado correctamente: {'ts': 1712203200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30110, 'Load(W)': 30110}}
Dato enviado correctamente: {'ts': 1712203500000.0, 'values': {'PV(W)': 0, 'Meter(W)': -

Dato enviado correctamente: {'ts': 1712197500000.0, 'values': {'PV(W)': 0, 'Meter(W)': -26385, 'Load(W)': 26385}}
Dato enviado correctamente: {'ts': 1712197800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -28713, 'Load(W)': 28713}}
Dato enviado correctamente: {'ts': 1712198100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -28713, 'Load(W)': 28713}}
Dato enviado correctamente: {'ts': 1712198400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -28338, 'Load(W)': 28338}}
Dato enviado correctamente: {'ts': 1712198700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -26388, 'Load(W)': 26388}}
Dato enviado correctamente: {'ts': 1712199000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -25294, 'Load(W)': 25294}}
Dato enviado correctamente: {'ts': 1712199300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -24237, 'Load(W)': 24237}}
Dato enviado correctamente: {'ts': 1712199600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -23939, 'Load(W)': 23939}}
Dato enviado correctamente: {'ts': 1712199900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -

...

Dato enviado correctamente: {'ts': 1712194500000.0, 'values': {'PV(W)': 0, 'Meter(W)': -4002, 'Load(W)': 4002}}
Dato enviado correctamente: {'ts': 1712194800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -4182, 'Load(W)': 4182}}
Dato enviado correctamente: {'ts': 1712195100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -3684, 'Load(W)': 3684}}
Dato enviado correctamente: {'ts': 1712195400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -4872, 'Load(W)': 4872}}
Dato enviado correctamente: {'ts': 1712195700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -4782, 'Load(W)': 4782}}
Dato enviado correctamente: {'ts': 1712196000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -4710, 'Load(W)': 4710}}
Dato enviado correctamente: {'ts': 1712196300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -4305, 'Load(W)': 4305}}
Dato enviado correctamente: {'ts': 1712196600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -3900, 'Load(W)': 3900}}
Dato enviado correctamente: {'ts': 1712196900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -4083, 'Load(W)':

Dato enviado correctamente: {'ts': 1712216400000.0, 'values': {'PV(W)': 6827, 'Meter(W)': -9694, 'Load(W)': 16521}}
Dato enviado correctamente: {'ts': 1712216700000.0, 'values': {'PV(W)': 12665, 'Meter(W)': -4263, 'Load(W)': 16928}}
Dato enviado correctamente: {'ts': 1712217000000.0, 'values': {'PV(W)': 13092, 'Meter(W)': -6045, 'Load(W)': 19137}}
Dato enviado correctamente: {'ts': 1712217300000.0, 'values': {'PV(W)': 18302, 'Meter(W)': -4101, 'Load(W)': 22403}}
Dato enviado correctamente: {'ts': 1712217600000.0, 'values': {'PV(W)': 20103, 'Meter(W)': -3717, 'Load(W)': 23820}}
Dato enviado correctamente: {'ts': 1712217900000.0, 'values': {'PV(W)': 17181, 'Meter(W)': -5778, 'Load(W)': 22959}}
Dato enviado correctamente: {'ts': 1712218200000.0, 'values': {'PV(W)': 16436, 'Meter(W)': -8333, 'Load(W)': 24769}}
Dato enviado correctamente: {'ts': 1712218500000.0, 'values': {'PV(W)': 0, 'Meter(W)': 3600, 'Load(W)': 0}}
Dato enviado correctamente: {'ts': 1712218800000.0, 'values': {'PV(W)': 48

Dato enviado correctamente: {'ts': 1712188200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -14949, 'Load(W)': 14949}}
Dato enviado correctamente: {'ts': 1712188500000.0, 'values': {'PV(W)': 0, 'Meter(W)': -14778, 'Load(W)': 14778}}
Dato enviado correctamente: {'ts': 1712188800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -14013, 'Load(W)': 14013}}
Dato enviado correctamente: {'ts': 1712189100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -14490, 'Load(W)': 14490}}
Dato enviado correctamente: {'ts': 1712189400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -14685, 'Load(W)': 14685}}
Dato enviado correctamente: {'ts': 1712189700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -12396, 'Load(W)': 12396}}
Dato enviado correctamente: {'ts': 1712190000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -15693, 'Load(W)': 15693}}
Dato enviado correctamente: {'ts': 1712190300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -13707, 'Load(W)': 13707}}
Dato enviado correctamente: {'ts': 1712190600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -

Dato enviado correctamente: {'ts': 1712209800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -39171, 'Load(W)': 39171}}
Dato enviado correctamente: {'ts': 1712210100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -34140, 'Load(W)': 34140}}
Dato enviado correctamente: {'ts': 1712210400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -34365, 'Load(W)': 34365}}
Dato enviado correctamente: {'ts': 1712210700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -32460, 'Load(W)': 32460}}
Dato enviado correctamente: {'ts': 1712211000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -35895, 'Load(W)': 35895}}
Dato enviado correctamente: {'ts': 1712211300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -35838, 'Load(W)': 35838}}
Dato enviado correctamente: {'ts': 1712211600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -36276, 'Load(W)': 36276}}
Dato enviado correctamente: {'ts': 1712211900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -36324, 'Load(W)': 36324}}
Dato enviado correctamente: {'ts': 1712212200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -

Dato enviado correctamente: {'ts': 1712231100000.0, 'values': {'PV(W)': 66455, 'Meter(W)': 1743, 'Load(W)': 64712}}
Dato enviado correctamente: {'ts': 1712231400000.0, 'values': {'PV(W)': 56678, 'Meter(W)': -2787, 'Load(W)': 59465}}
Dato enviado correctamente: {'ts': 1712231700000.0, 'values': {'PV(W)': 63681, 'Meter(W)': 3231, 'Load(W)': 60450}}
Dato enviado correctamente: {'ts': 1712232000000.0, 'values': {'PV(W)': 62990, 'Meter(W)': 4764, 'Load(W)': 58226}}
Dato enviado correctamente: {'ts': 1712232300000.0, 'values': {'PV(W)': 62299, 'Meter(W)': 4437, 'Load(W)': 57862}}
Dato enviado correctamente: {'ts': 1712232600000.0, 'values': {'PV(W)': 59948, 'Meter(W)': -6954, 'Load(W)': 66902}}
Dato enviado correctamente: {'ts': 1712232900000.0, 'values': {'PV(W)': 61482, 'Meter(W)': 5607, 'Load(W)': 55875}}
Dato enviado correctamente: {'ts': 1712233200000.0, 'values': {'PV(W)': 60633, 'Meter(W)': 3948, 'Load(W)': 56685}}
Dato enviado correctamente: {'ts': 1712233500000.0, 'values': {'PV(W)'